#   LangChain Tool 활용

---

## 환경 설정 및 준비

`(1) Env 환경변수`

In [ ]:
from dotenv import load_dotenv
load_dotenv()

`(2) 기본 라이브러리`

In [ ]:
import os
from glob import glob

from pprint import pprint
import json

`(3) Langsmith tracing 설정`

In [ ]:
# Langsmith tracing 여부를 확인 (true: langsmith 추척 활성화, false: langsmith 추척 비활성화)
import os
print(os.getenv('LANGSMITH_TRACING'))

---

## **내장 도구 (Built-in Tool)**

- **내장 도구**는 시스템에 **기본적으로 포함**된 사전 정의된 도구들의 집합

- 파일 처리, 웹 검색, 코드 실행 등 **자주 사용되는 기본 기능**들을 즉시 활용 가능

- 별도의 구현 없이 **바로 사용**할 수 있어 개발 시간과 노력을 절약 가능

---

### **웹 검색(Tavily Search API)**

- **Tavily**는 AI 에이전트(LLM)를 위해 특별히 설계된 검색 엔진

- 검색 결과로 **제목, URL, 컨텐츠, 답변** 등 상세 정보 제공

- 개발자는 **월 1,000회** 무료 검색 쿼터 사용 가능

- **설치**: `pip install tavily-python` 또는 `uv add tavily-python`

- **인증키**: `TAVILY_API_KEY`

`(1) 도구 정의`

In [ ]:
from langchain_community.tools import TavilySearchResults

tool = TavilySearchResults(
    max_results=5,  # 반환할 결과의 수
    search_depth="advanced",  # 검색 깊이: basic 또는 advanced
    include_answer=True,  # 결과에 직접적인 답변 포함
    include_raw_content=True,  # 페이지의 원시 콘텐츠 포함
    include_images=True,  # 결과에 이미지 포함
    # include_domains=[...],  # 특정 도메인으로 검색 제한
    # exclude_domains=[...],  # 특정 도메인 제외
    # name="...",  # 기본 도구 이름 덮어쓰기
    # description="...",  # 기본 도구 설명 덮어쓰기
    # args_schema=...,  # 기본 args_schema 덮어쓰기
)

`(2) 도구 직접 호출`
- 자연어 쿼리를 도구에 전달

In [ ]:
result = tool.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
pprint(result)

`(3) ToolCall을 사용한 호출`
- 모델에서 생성된 `ToolCall`을 사용하여 도구를 호출

In [ ]:
from langchain_openai import ChatOpenAI

# 모델 생성
model = ChatOpenAI(model="gpt-4.1-mini")

# 모델에 도구 등록 
model_with_tools = model.bind_tools([tool])

# 사용자 쿼리를 입력하여 ToolCall 생성
response = model_with_tools.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")

# ToolCall 출력 
pprint(response.tool_calls)

In [ ]:
# ToolCall을 사용하여 도구 실행
model_generated_tool_call = response.tool_calls[0]
tool_msg = tool.invoke(model_generated_tool_call)

# 도구 실행 결과 출력
pprint(tool_msg.content)

In [ ]:
# `artifact` 속성 출력
tool_msg.artifact

In [ ]:
pprint(tool_msg.artifact)

`(4) LLM 체인과 연계`
- Tavily 도구를 언어 모델에 바인딩하고 사용자 입력을 처리하는 체인을 생성

In [ ]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableConfig, chain
from langchain_core.messages import ToolMessage

import datetime

# 언어 모델 초기화
llm = ChatOpenAI(model="gpt-4.1-mini")

# 프롬프트 템플릿 정의
today = datetime.datetime.today().strftime("%Y-%m-%d")
prompt = ChatPromptTemplate(
    [
        ("system", f"당신은 도움이 되는 어시스턴트입니다. 오늘 날짜는 {today}입니다."),
        ("human", "{user_input}"),
        ("placeholder", "{messages}"),
    ]
)

# Tavily 도구를 모델에 바인딩
llm_with_tools = llm.bind_tools([tool])

# 체인 생성
llm_chain = prompt | llm_with_tools

@chain  # 함수를 체인으로 사용하기 위해 데코레이터 추가
def tool_chain(user_input: str, config: RunnableConfig):
    input_ = {"user_input": user_input}
    ai_msg = llm_chain.invoke(input_, config=config)
    
    # 도구 호출 결과를 처리
    tool_msgs = []
    artifacts = []  # artifact를 저장할 리스트
    for tool_call in ai_msg.tool_calls:
        tool_result = tool.invoke(tool_call)
        tool_msgs.append(ToolMessage(content=str(tool_result.content), tool_call_id=tool_call["id"]))
        artifacts.append(tool_result.artifact)  # artifact 저장
    
    # 최종 결과 반환
    final_response = llm_chain.invoke({**input_, "messages": [ai_msg, *tool_msgs]}, config=config)
    
    return final_response, artifacts

# 체인 호출
response, artifacts = tool_chain.invoke("한국 시장에서 거래되는 ETF 종목은 모두 몇 개인가요?")
print("===== 최종 응답 =====")
print(response.content)

In [ ]:
# artifact를 별도로 출력
print("===== Artifact 정보 =====")
for artifact in artifacts:
    print(json.dumps(artifact, indent=2, ensure_ascii=False))

---

##  **사용자 정의 도구 (Custom Tool)**


- **사용자 정의 도구**는 개발자가 직접 설계하고 구현하는 **맞춤형 함수나 도구**를 의미

- LLM이 호출할 수 있는 **고유한 기능**을 정의하여 특정 작업에 최적화된 도구 생성 가능

- 개발자는 도구의 **입력값, 출력값, 기능**을 자유롭게 정의하여 유연한 확장성 확보

---

### **Naver 개발자 API** 를 도구로 사용

- 네이버 개발자 API(https://developers.naver.com/)에서 인증 권한 취득 (회원 가입 및 애플리케이션 등록 필요)
- 환경변수(.env)를 등록합니다. (**NAVER_CLIENT_ID**, **NAVER_CLIENT_SECRET**)
- 아래 정의된 네이버 뉴스 검색 도구(naver_news_search)를 사용하여 다음 과정을 수행합니다. 

In [ ]:
# 환경 변수 로드
from dotenv import load_dotenv
load_dotenv()

`(1) 도구 정의`

In [ ]:
import requests, os
from langchain_core.tools import tool
from typing import Dict

@tool
def naver_news_search(
    query: str,
    ) -> Dict[Dict, int]:
    """네이버 검색 API를 사용하여 뉴스 검색 결과를 조회합니다."""

    url = "https://openapi.naver.com/v1/search/news.json"
    headers = {
        "X-Naver-Client-Id": os.getenv("NAVER_CLIENT_ID"),
        "X-Naver-Client-Secret": os.getenv("NAVER_CLIENT_SECRET")
    }
    params = {"query": query}

    response = requests.get(url, headers=headers, params=params)

    return {
        "data": response.json(),
        "status_code": int(response.status_code)
    }  #type: ignore

`(2) 도구 실행`

In [ ]:
from langchain_core.runnables import chain, RunnableLambda
from langchain_openai import ChatOpenAI

# 도구 맵 생성 (도구 이름을 키로 사용)
tools = [naver_news_search]  
tool_map = {tool.name: tool for tool in tools}

# 도구 라우터 (도구 이름에 따라 실행할 도구를 선택)
@chain
def tool_router(tool_call):
    return tool_map[tool_call["name"]]

# 체인 구성
llm = ChatOpenAI(model="gpt-4.1-mini", temperature=0)

llm_with_tools = llm.bind_tools(tools)

tool_chain = (
    llm_with_tools    # 어떤 도구를 사용할지 결정 (LLM 모델이 도구 호출을 처리)
    | RunnableLambda(lambda x: x.tool_calls)  # 도구 호출을 추출
    | tool_router.map()   # 도구 호출 라우팅
)
# 도구 실행
response = tool_chain.invoke("애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")

In [ ]:
# 결과 출력 (도구 호출 결과)
for msg in response:
    pprint(msg)

`(3) 체인 실행`

In [ ]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import chain
from langchain_core.messages import ToolMessage
from langchain_openai import ChatOpenAI
from datetime import datetime

# LLM 모델 인스턴스를 생성
llm = ChatOpenAI(model="gpt-4.1-mini")

# 두 도구를 LLM에 바인딩
llm_with_tools = llm.bind_tools(tools=[naver_news_search])

# 오늘 날짜 설정
today = datetime.today().strftime("%Y-%m-%d")

# 프롬프트 템플릿 정의
prompt = ChatPromptTemplate([
    ("system", f"당신은 도움이 되는 AI 어시스턴트입니다. 오늘 날짜는 {today}입니다. 답변 생성의 근거 또는 출처를 명시하세요."),
    ("human", "{user_input}"),  
    ("placeholder", "{messages}"),
])

# LLM 체인 생성
llm_chain = prompt | llm_with_tools

@chain
def news_analysis_chain(user_input: str):

    # 도구 체인 실행 (LLM -> StockPrice/NaverNewsSearch)
    tool_msgs = tool_chain.invoke(user_input)

    print(f"Tool Messages: {tool_msgs}")

    # 도구 결과를 포맷팅
    formatted_messages = []
    for msg in tool_msgs:
        if isinstance(msg, ToolMessage) and len(msg.content) > 0:
            formatted_messages.append({"role": "assistant", "content": msg.content}) 

    print(f"Formatted Messages: {formatted_messages}")

    # 최종 응답 생성
    final_response = llm_chain.invoke({"user_input": user_input, "messages": formatted_messages})

    print(f"Final Response: {final_response}")

    return final_response

In [ ]:
# 체인 실행
response = news_analysis_chain.invoke("애플의 최근 주가는 어떻게 되나요? 최근 주가 분석에 대한 관련 기사를 찾아주세요.")

In [ ]:
# 최종 응답 출력 
print(response.content)